In [ ]:
# Jhose Immanuel Sembiring
# 1103202047
# TK44G4
# Neural_Network_Predicting-House-Price-Kaggle_GColabs
# Dokumentasi ChatGPT : https://chatgpt.com/share/bfabd051-76ad-4c6d-88d7-c82649ef8b39

#Menghubungkan ke GDrive


In [3]:
# Import library yang diperlukan
from google.colab import drive

# Mount Google Drive ke Colab
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Library

In [33]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor




#Menampilkan Dataframe

In [19]:
# Path ke file di Google Drive
file_path = '/content/drive/MyDrive/Semester8/Machine Learning/lecture 10/dataset/train.csv'

# Membaca file CSV ke DataFrame
df = pd.read_csv(file_path)

# Menampilkan beberapa baris pertama
df


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [20]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [21]:
df.shape

(1460, 81)

#Data Preprocessing

In [22]:
# Mengisi nilai yang hilang
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns

# Mengisi nilai yang hilang pada fitur numerik dengan median
for col in numerical_features:
    df[col].fillna(df[col].median(), inplace=True)

# Mengisi nilai yang hilang pada fitur kategorikal dengan modus
for col in categorical_features:
    df[col].fillna(df[col].mode()[0], inplace=True)


In [ ]:
Mengubah Fitur Kategorikal Menjadi Numerik
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)


In [26]:
#Normalisasi Fitur Numerik
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])


In [27]:
# Membagi dataset menjadi fitur dan target
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

print("Data preprocessing selesai.")
print("X shape:", X.shape)
print("y shape:", y.shape)

Data preprocessing selesai.
X shape: (1460, 245)
y shape: (1460,)


#Error Measure

In [29]:
# Membagi dataset menjadi training dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model regresi linear
model = LinearRegression()
model.fit(X_train, y_train)

# Melakukan prediksi pada testing set
y_pred = model.predict(X_test)

# Menghitung MSE dan RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')


Mean Squared Error (MSE): 8.878326779774384e+18
Root Mean Squared Error (RMSE): 2979652123.9524565


# K-Fold Cross-Validation

In [31]:
# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Menggunakan 5 fold
rmse_scores = []

#Model Selection

In [34]:
# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Menggunakan 5 fold

# Daftar model untuk diuji
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42)
}

# Dictionary untuk menyimpan RMSE setiap model
model_rmse = {}

for model_name, model in models.items():
    rmse_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        rmse_scores.append(rmse)

    average_rmse = np.mean(rmse_scores)
    model_rmse[model_name] = average_rmse
    print(f'{model_name} - Average RMSE: {average_rmse}')

# Menentukan model dengan RMSE terendah
best_model_name = min(model_rmse, key=model_rmse.get)
best_model_rmse = model_rmse[best_model_name]

print(f'\nBest Model: {best_model_name} with Average RMSE: {best_model_rmse}')

Linear Regression - Average RMSE: 2266795083.9396386
Decision Tree Regressor - Average RMSE: 0.5448426601483207
Random Forest Regressor - Average RMSE: 0.38455698847910624
Gradient Boosting Regressor - Average RMSE: 0.3684371354055073

Best Model: Gradient Boosting Regressor with Average RMSE: 0.3684371354055073
